In [1]:
import numpy as np
from iminuit import Minuit
from scipy.integrate import quad

import constants

def dL(z, omega, H0):
    borne = (omega/(1-omega)) ** (1/3)
    X = 1 / (H0 * np.sqrt(1-omega))
    def integrale(z_i):
        res, _ = quad(lambda t: 1/np.sqrt(1 + t**3), borne, borne * (1+z_i))
        return constants.C * (1+z_i) * X * res
    return np.array([integrale(z_i) for z_i in np.atleast_1d(z)])

def mu(z, omega, H0):
    # Vérifie l'unité de dL !
    return 5 * np.log10(dL(z, omega, H0)) + 25

def chi2(omega, H0):
    delta_mu = mu(constants.z_data_panth, omega, H0) - constants.mu_data_panth
    variances = constants.cov_mat.to_numpy().flatten()
    return np.sum((delta_mu**2) / variances)

In [2]:
c = constants.cov_mat.to_numpy().flatten()
np.min(c)

np.float64(-0.03172511)

In [4]:
n_om = 100
om_min = .05
om_max = 1

n_s8 = 100
s8_min = 0.4
s8_max = 1.2

n_gamma = 100
gamma_min = -0.5
gamma_max = 2

omega_vals = np.asarray(np.linspace(om_min, om_max, n_om))
sigma_vals = np.asarray(np.linspace(s8_min, s8_max, n_s8))
gamma_vals = np.asarray(np.linspace(gamma_min, gamma_max, n_gamma))

minimizer = Minuit(chi2, omega=0.315, H0=70)

minimizer.limits["omega"] = (om_min,om_max)
# minimizer.limits["sigma"] = (s8_min,s8_max)
# minimizer.limits["gamma"] = (gamma_min,gamma_max)

result = minimizer.migrad()

print(f"\nRésultats du fit :")
print(f"Chi2 = {minimizer.fval:.2f} \n")
print(f"Paramètres estimés :")
print(f"Ωm = {minimizer.values['omega']:.3f} ± {minimizer.errors['omega']:.4f}")
print(f"H0 = {minimizer.values['H0']:.3f} ± {minimizer.errors['h0']:.4f}")
# print(f"σ8,0 = {minimizer.values['sigma']:.3f} ± {minimizer.errors['sigma']:.4f}")
# print(f"γ = {minimizer.values['gamma']:.3f} ± {minimizer.errors['gamma']:.3f}\n")

print(result)

ValueError: operands could not be broadcast together with shapes (1701,) (2893401,) 

In [32]:
from scipy.special import ellipkinc
from scipy.integrate import quad
import numpy as np

from matplotlib import pyplot as plt

def primitive(x):
    x = np.asarray(x)
    assert np.all(x > 0), "Cette fonction n'est définie que pour x > 0"
    phi = np.arcsin(np.sqrt((x * (x + 1)) / (x + (x + 1)**2)))
    k = 1 / np.sqrt(2)
    return 2 / np.sqrt(x) * ellipkinc(phi, k)

# Vérification
x = 0.1
print("Primitive (elliptique) :", primitive(x))
print("Primitive (quad)       :", quad(lambda t: 1/np.sqrt(1 + t**3), 0, x)[0])


Primitive (elliptique) : 1.8784991938778408
Primitive (quad)       : 0.09998750535401997
